In [6]:
import pandas as pd
import json
def load_config(path):
    with open(path, 'r') as file:
        return json.load(file)

config_path = '../pandas_config.json'
config = load_config(config_path)
parquet_config = config['parquet']

# Setting up the engine as a global constant
ENGINE = parquet_config['engine']
COMPRESSION = parquet_config['compression']
FILE_PATH = "../data/data_parquet"

In [8]:
df = pd.read_parquet(FILE_PATH + "/processed_data_combined.parquet", engine=ENGINE)

# Ratio: Actions performed by humans and service accounts

In [13]:
user_ratio = df["user_name"].value_counts(normalize=True).to_frame().to_csv("../data/visualization/user_ratio.csv")

# Airports with most departures

In [12]:
df_unique_flight_numbers = df.drop_duplicates(subset='flight_number')
count_of_departures_by_airport = df_unique_flight_numbers[["departure_airport", "departure_lat", "departure_lon"]].value_counts().to_frame().to_csv("../data/visualization/count_of_departures_by_airport.csv")